In [1]:
from utils import aws, utils
from ast import literal_eval
from transformers import AutoModelForCausalLM, AutoTokenizer

/Users/nace/local/conda/envs/hf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/nace/local/conda/envs/hf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/nace/local/conda/envs/hf/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  F

In [ ]:
hf_llama_token = literal_eval(aws.get_secret("prod/HuggingFace/key"))["hf-llama-token"]

In [ ]:
# Define the model and tokenizer
repo_id = "meta-llama/Llama-3.1-8B-Instruct"

# Download the tokenizer
tokenizer = AutoTokenizer.from_pretrained(repo_id, token=hf_llama_token)

In [ ]:
# Download the model
model = AutoModelForCausalLM.from_pretrained(repo_id, torch_dtype="auto", token=hf_llama_token)

In [ ]:
model

In [ ]:
# Define local save paths
model_dir = "./llama_model"

# Save the model and tokenizer locally
tokenizer.save_pretrained(model_dir)
model.save_pretrained(model_dir)

In [ ]:
local_model_dir = "./llama_model"
tokenizer_2 = AutoTokenizer.from_pretrained(local_model_dir)
model_2 = AutoModelForCausalLM.from_pretrained(local_model_dir, torch_dtype="auto")

In [ ]:
model_2

In [8]:
import boto3
import os

# Set your bucket and prefix
bucket_name = "esg-portfolio-bucket"
prefix = "ml/meta-llama/Llama-3.1-8B-Instruct/"
model_dir = "llama_model"

# Initialize S3 client
s3 = boto3.client("s3")
files_upload = [
    #"model-00001-of-00004.safetensors",
    #"model-00002-of-00004.safetensors",
    "model-00003-of-00004.safetensors",
]

# Upload files to S3
def upload_directory_to_s3(local_directory, s3_bucket, s3_prefix):
    for root, _, files in os.walk(local_directory):
        for file in files:
            if file in files_upload:
                local_path = os.path.join(root, file)
                relative_path = os.path.relpath(local_path, local_directory)
                s3_path = os.path.join(s3_prefix, relative_path)

                print(f"Uploading {local_path} to s3://{s3_bucket}/{s3_path}")
                s3.upload_file(local_path, s3_bucket, s3_path)

upload_directory_to_s3(model_dir, bucket_name, prefix)


Uploading llama_model/model-00003-of-00004.safetensors to s3://esg-portfolio-bucket/ml/meta-llama/Llama-3.1-8B-Instruct/model-00003-of-00004.safetensors
